In [81]:
import math
class Value:
    
    def __init__(self,data,label='',_children=(),_op=''):
        self.data = data
        self.grad = 0.0 
        self.lable = label
        self._backward = lambda:None
        self._prev = set(_children)
        self._op = _op
 
    def __repr__(self):
        return f"Value(data = {self.data})"
    
    def __add__(self,other):
        other = other if isinstance(other,Value) else Value(other)
        out = Value(self.data + other.data,(self,other),'+')
        
        def _backward():
            self.grad += out.grad 
            other.grad += out.grad
        out._backward = _backward
        
        return out


    def __sub__(self,other):
        other = other if isinstance(other,Value) else Value(other)
        out = Value(self.data - other.data,(self,other),'-')

        def _backward():
            self.grad += out.grad 
            other.grad += out.grad
        out._backward = _backward
        
        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        
        return out
        
    def __truediv__(self, other): # self / other
        return self * other**-1
        
        
    def __radd__(self, other): # other + self
        return self + other
    
    def __neg__(self): # -self
        return self * -1
    
    def __rsub__(self,other):
        self.data*=-1
        other*=-1
        return self - other
    
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ), 'exp')
    
        def _backward():
            self.grad += out.data * out.grad # NOTE: in the video I incorrectly used = instead of +=. Fixed here.
        out._backward = _backward
    
        return out
    
    def __rmul__(self, other): # other * self
        return self * other
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, (self, ), 'tanh')
        
        def _backward():
            self.grad += (1 - t**2) * out.grad
        
        out._backward = _backward
        return out
    
    def backward(self):
        
        topo = []
        visited = set()
        def build_topo(v):               
            if v not in visited: 
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                    topo.append(v)
            build_topo(self)
            topo
            self.grad = 1.0
            for node in reversed(topo):
                  node._backward()

In [82]:
import random

class Neuron:
  
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))

    def __call__(self, x):
        # w * x + b
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out

    def parameters(self):
        return self.w + [self.b]

class Layer:
  
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs

    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:  
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]                                              
    
    

In [83]:
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(x)


Value(data = 0.8636253289969622)

In [84]:
n.parameters()

[Value(data = -0.15896236884806902),
 Value(data = -0.6931889353601468),
 Value(data = 0.11854664466217057),
 Value(data = 0.12730100922568188),
 Value(data = 0.4548447358663834),
 Value(data = 0.13816042782273796),
 Value(data = 0.38182867509085705),
 Value(data = -0.1498750645868081),
 Value(data = 0.17278380445320463),
 Value(data = -0.42441009524277606),
 Value(data = 0.5174267053694159),
 Value(data = -0.3072886040856475),
 Value(data = 0.8415576290592559),
 Value(data = 0.7935814253229696),
 Value(data = 0.34289067118758854),
 Value(data = -0.942857475798857),
 Value(data = -0.9510963752666748),
 Value(data = 0.5452502731948834),
 Value(data = -0.0629838085023926),
 Value(data = 0.03128296470930647),
 Value(data = -0.7553180963589234),
 Value(data = -0.23960144617536439),
 Value(data = 0.40169745105014587),
 Value(data = 0.709148813780361),
 Value(data = 0.6911837929220261),
 Value(data = -0.3497060281687776),
 Value(data = -0.017483513135609474),
 Value(data = 0.4818942475490666

In [85]:
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] # desired targets
ypred =  [n(x) for x in xs]
ypred

[Value(data = 0.8636253289969622),
 Value(data = 0.9314650877341198),
 Value(data = 0.8375257391726898),
 Value(data = 0.5462663594096082)]

In [86]:
loss=sum((yout.data - ygt)**2 for ygt, yout in zip(ys, ypred))
loss = Value(loss)

In [87]:
loss.backward()

In [89]:
n.layers[0].neurons[0].w[0].grad

0.0

In [78]:
for k in range(20):
  
    # forward pass
    ypred = [n(x) for x in xs]
    loss=Value(sum((yout.data - ygt)**2 for ygt, yout in zip(ys, ypred)))
    
    # backward pass
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()

    # update
    for p in n.parameters():
        p.data += -0.1 * p.grad

    print(k, loss.data)

0 6.3465736162105335
1 6.3465736162105335
2 6.3465736162105335
3 6.3465736162105335
4 6.3465736162105335
5 6.3465736162105335
6 6.3465736162105335
7 6.3465736162105335
8 6.3465736162105335
9 6.3465736162105335
10 6.3465736162105335
11 6.3465736162105335
12 6.3465736162105335
13 6.3465736162105335
14 6.3465736162105335
15 6.3465736162105335
16 6.3465736162105335
17 6.3465736162105335
18 6.3465736162105335
19 6.3465736162105335


In [ ]:
*---------------------------------------------------*